In [ ]:
import sagemaker
import boto3

In [ ]:
sess = sagemaker.Session()
sess

In [ ]:
role = sagemaker.get_execution_role()
role

In [ ]:
sess.boto_region_name

In [ ]:

region_mapping = {
    "af-south-1": "626614931356",
    "il-central-1": "780543022126",
    "ap-east-1": "871362719292",
    "ap-northeast-1": "763104351884",
    "ap-northeast-2": "763104351884",
    "ap-northeast-3": "364406365360",
    "ap-south-1": "763104351884",
    "ap-south-2": "772153158452",
    "ap-southeast-1": "763104351884",
    "ap-southeast-2": "763104351884",
    "ap-southeast-3": "907027046896",
    "ap-southeast-4": "457447274322",
    "ca-central-1": "763104351884",
    "cn-north-1": "727897471807",
    "cn-northwest-1": "727897471807",
    "eu-central-1": "763104351884",
    "eu-central-2": "380420809688",
    "eu-north-1": "763104351884",
    "eu-west-1": "763104351884",
    "eu-west-2": "763104351884",
    "eu-west-3": "763104351884",
    "eu-south-1": "692866216735",
    "eu-south-2": "503227376785",
    "me-south-1": "217643126080",
    "me-central-1": "914824155844",
    "sa-east-1": "763104351884",
    "us-east-1": "763104351884",
    "us-east-2": "763104351884",
    "us-gov-east-1": "446045086412",
    "us-gov-west-1": "442386744353",
    "us-iso-east-1": "886529160074",
    "us-isob-east-1": "094389454867",
    "us-west-1": "763104351884",
    "us-west-2": "763104351884",
}

llm_image = f"{region_mapping[sess.boto_region_name]}.dkr.ecr.{sess.boto_region_name}.amazonaws.com/huggingface-pytorch-tgi-inference:2.1.1-tgi1.3.3-gpu-py310-cu121-ubuntu20.04-v1.0"

print(f"llm image uri: {llm_image}")

In [ ]:
s3_uri = huggingface_estimator.model_data
print(s3_uri)

In [ ]:
import json
from sagemaker.huggingface import HuggingFaceModel


instance_type = "ml.g5.48xlarge"
number_of_gpu = 8
health_check_timeout = 300

config = {
    "HF_MODEL_ID": "/opt/ml/model",
    "SM_NUM_GPUS": json.dumps(number_of_gpu),
    "MAX_INPUT_LENGTH": json.dumps(24000), 
    "MAX_BATCH_PREFILL_TOKENS": json.dumps(32000),
    "MAX_TOTAL_TOKENS": json.dumps(32000),
    "MAX_BATCH_TOTAL_TOKENS": json.dumps(
        512000
    ),
}


llm_model = HuggingFaceModel(model_data=s3_uri, role=role, image_uri=llm_image, env=config)

In [ ]:


endpoint_name = sagemaker.utils.name_from_base("Mixtral-8x7B")

llm = llm_model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
)

In [ ]:

prompt = "What is Amazon SageMaker?"


payload = {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.1,
    "top_k": 50,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "return_full_text": False,
    "stop": ["</s>"],
}

In [ ]:
chat = llm.predict({"inputs": prompt, "parameters": payload})

print(chat[0]["generated_text"])